## PREPROCESSING

### Install and Importing Library/Package

In [3]:
%pip install nltk


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import plotly.express as px

from sklearn.preprocessing import LabelEncoder

# Import function for preprocessing
import function as fc

### Gathering Data

In [62]:
# Read datasets
data_train = pd.read_csv('../datasets/raw/train.txt', names=['Text', 'Emotion'], sep=';')
data_val = pd.read_csv('../datasets/raw/val.txt', names=['Text', 'Emotion'], sep=';')
data_test = pd.read_csv('../datasets/raw/test.txt', names=['Text', 'Emotion'], sep=';')

In [63]:
data_train = data_train.rename(columns={'Text': 'text', 'Emotion': 'label'})
data_val = data_val.rename(columns={'Text': 'text', 'Emotion': 'label'})
data_test = data_test.rename(columns={'Text': 'text', 'Emotion': 'label'})

In [64]:
#print first 5 rows
data_train.head()

,text,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [65]:
#print the shape of the data set
print(data_train.shape)

(16000, 2)


In [66]:
#print first 5 rows
data_val.head()

,text,label
0,im feeling quite sad and sorry for myself but ...,sadness
1,i feel like i am still looking at a blank canv...,sadness
2,i feel like a faithful servant,love
3,i am just feeling cranky and blue,anger
4,i can have for a treat or if i am feeling festive,joy


In [67]:
#print the shape of the data set
print(data_val.shape)

(2000, 2)


In [68]:
#print first 5 rows
data_test.head()

,text,label
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness


In [69]:
#print the shape of the data set
print(data_test.shape)

(2000, 2)


### Assessing Data

#### Melihat informasi sederhana dari data

In [70]:
# Melihat informasi sederhana dari data_train
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    16000 non-null  object
 1   label   16000 non-null  object
dtypes: object(2)
memory usage: 250.1+ KB


In [71]:
# Melihat informasi sederhana dari data_val
data_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2000 non-null   object
 1   label   2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [72]:
# Melihat informasi sederhana dari data_test
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2000 non-null   object
 1   label   2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


#### Melihat Missing Value

In [73]:
print("\nJumlah Nan Value pada data_train \n",data_train.isna().sum())
print("\nJumlah Nan Value pada data_val \n",data_val.isna().sum())
print("\nJumlah Nan Value pada data_test \n",data_test.isna().sum())


Jumlah Nan Value pada data_train 
 text     0
label    0
dtype: int64

Jumlah Nan Value pada data_val 
 text     0
label    0
dtype: int64

Jumlah Nan Value pada data_test 
 text     0
label    0
dtype: int64


#### Mengecek duplikat data

In [74]:
print("\nJumlah Duplikat Data pada data_train \n",data_train.duplicated().sum())
print("\nJumlah Duplikat Data pada data_val \n",data_val.duplicated().sum())
print("\nJumlah Duplikat Data pada data_test \n",data_test.duplicated().sum())


Jumlah Duplikat Data pada data_train 
 1

Jumlah Duplikat Data pada data_val 
 0

Jumlah Duplikat Data pada data_test 
 0


In [75]:
print("\nJumlah Duplikat Data pada data_train \n",data_train['text'].duplicated().sum())


Jumlah Duplikat Data pada data_train 
 31


#### Mengecek detail di setiap kolom dataframe

In [76]:
data_train.describe()

,text,label
count,16000,16000
unique,15969,6
top,i feel on the verge of tears from weariness i ...,joy
freq,2,5362


In [77]:
data_val.describe()

,text,label
count,2000,2000
unique,1998,6
top,i feel so tortured by it,joy
freq,2,704


In [78]:
data_test.describe()

,text,label
count,2000,2000
unique,2000,6
top,im feeling rather rotten so im not very ambiti...,joy
freq,1,695


In [79]:
#print the rows which are duplicated (duplicated in the text but with different emotions)
data_train[data_train['text'].duplicated() == True]

,text,label
5067,i feel on the verge of tears from weariness i ...,joy
6133,i still feel a craving for sweet food,love
6563,i tend to stop breathing when i m feeling stre...,anger
7623,i was intensely conscious of how much cash i h...,sadness
7685,im still not sure why reilly feels the need to...,surprise
8246,i am not amazing or great at photography but i...,love
9596,ive also made it with both sugar measurements ...,joy
9687,i had to choose the sleek and smoother feel of...,joy
9769,i often find myself feeling assaulted by a mul...,sadness
9786,i feel im being generous with that statement,joy


In [80]:
#print the rows which are duplicated (duplicated in the text but with different emotions)
data_val[data_val['text'].duplicated() == True]

,text,label
603,i have had several new members tell me how com...,joy
1993,i feel so tortured by it,anger


In [81]:
#print the rows which are duplicated (duplicated in the text but with different emotions)
data_test[data_test['text'].duplicated() == True]

,text,label


### Cleaning Data

#### Melakukan Drop Duplikat Data

In [82]:
print("\nJumlah Duplikat Data pada data_train \n",data_train.duplicated().sum())


Jumlah Duplikat Data pada data_train 
 1


In [83]:
data_train.drop_duplicates(inplace=True)
data_train.reset_index(drop=True, inplace=True)

In [84]:
#removing duplicated text
data_train.drop_duplicates(subset=['text'], inplace=True)
data_train.reset_index(drop=True, inplace=True)

In [85]:
#removing duplicated text
data_val.drop_duplicates(subset=['text'], inplace=True)
data_val.reset_index(drop=True, inplace=True)

### EDA

#### Mengecek distribusi data di setiap label

In [86]:
data_train.label.value_counts() *100/ len(data_train)

label
joy         33.502411
sadness     29.206588
anger       13.494896
fear        12.104703
love         8.134511
surprise     3.556891
Name: count, dtype: float64

In [88]:
fig = px.histogram(data_train, x="label", title="Distribusi Emosi")
fig.show()


In [89]:
data_val.label.value_counts() *100/ len(data_val)

label
joy         35.185185
sadness     27.527528
anger       13.713714
fear        10.610611
love         8.908909
surprise     4.054054
Name: count, dtype: float64

In [90]:
fig = px.histogram(data_val, x="label", title="Distribusi Emosi")
fig.show()


In [91]:
data_test.label.value_counts() *100/ len(data_test)

label
joy         34.75
sadness     29.05
anger       13.75
fear        11.20
love         7.95
surprise     3.30
Name: count, dtype: float64

In [92]:
fig = px.histogram(data_test, x="label", title="Distribusi Emosi")
fig.show()


#### Mengecek distribusi stopwords dalam train

In [93]:
data_train['stop_words'] = data_train['text'].apply(lambda x: len([word for word in x.split() if word in stop_words]))
data_train.stop_words.value_counts()

stop_words
6     1149
4     1117
5     1114
8     1104
3     1084
9     1040
7     1038
10     924
2      815
11     796
12     716
13     660
14     597
15     519
16     454
1      416
17     371
18     308
19     290
20     231
21     221
22     174
23     152
24     118
25     102
26      81
0       79
27      58
28      48
29      43
30      40
31      30
32      21
33      17
34      10
37       8
38       7
36       6
39       5
40       2
35       2
42       1
41       1
Name: count, dtype: int64

In [94]:
fig = px.histogram(data_train, x="stop_words", title="Distribusi Stop Word dalam data train")
fig.show()

In [95]:
data_val['stop_words'] = data_val['text'].apply(lambda x: len([word for word in x.split() if word in stop_words]))
data_val.stop_words.value_counts()

stop_words
8     145
7     140
10    135
6     135
3     133
4     128
2     125
5     125
9     119
11    110
12     89
13     84
14     69
15     63
1      58
16     57
19     38
17     34
18     34
20     28
21     26
23     23
22     18
24     14
26     13
0      11
25     10
28      6
27      5
30      5
32      4
31      3
29      3
34      3
33      3
35      2
Name: count, dtype: int64

In [96]:
fig = px.histogram(data_val, x="stop_words", title="Distribusi Stop Word dalam data validasi")
fig.show()

In [97]:
data_test['stop_words'] = data_test['text'].apply(lambda x: len([word for word in x.split() if word in stop_words]))
data_test.stop_words.value_counts()

stop_words
3     153
6     153
8     143
5     141
4     139
7     132
2     111
10    104
9     104
11    103
12     96
13     80
14     77
18     66
15     58
16     56
1      36
19     36
17     34
22     24
20     24
21     21
25     18
24     16
23     13
27     11
26      9
28      8
0       8
31      7
29      6
30      4
32      4
33      3
34      1
37      1
Name: count, dtype: int64

In [98]:
fig = px.histogram(data_test, x="stop_words", title="Distribusi Stop Word dalam data uji")
fig.show()

### Mengaplikasikan fungsi normalisasi untuk teks

In [ ]:
data_train = normalize_text(data_train)
data_test = normalize_text(data_test)
data_val = normalize_text(data_val)

### Melakukan export csv hasil preprocessing

In [110]:
data_train.to_csv('../datasets/output_preprocess/data_train.csv', index=False)
data_test.to_csv('../datasets/output_preprocess/data_test.csv', index=False)
data_val.to_csv('../datasets/output_preprocess/data_val.csv', index=False)